In [1]:
from bs4 import BeautifulSoup
import requests
import json
import os
from csv import writer
from tqdm import tqdm, trange
from string import digits
from os.path import exists
from random import randrange
import time

In [2]:
class CityData:
    def __init__(self,name,state,title,description):
        self.name = name
        self.state = state
        self.title = title
        self.description = description
    def write_single_csv(self):
        filename = './data/' + self.state + '_documents.csv'
        if not exists(filename):
            with open(filename, "x") as file:
                file.write("Name,State,Title,Description\n")
        with open(filename, "a", encoding='utf-8') as file:
            file.write(self.info())
            
    def write_whole_csv(self):
        filename = 'documents.csv'
        if not exists(filename):
            with open(filename, "x") as file:
                file.write("Name,State,Title,Description\n")
        with open(filename, "a", encoding='utf-8') as file:
            file.write(self.info())

    def info(self):
        rep = '"' + '","'.join([self.name, self.state, self.title, self.description]) + '"\n'    
        return rep

In [3]:
def get_stateurl(source_url,soup):
    #get url linked to each state
    urls=soup.select('div[id="tabs_by_category"]>ul[class="tab-list tab-list-short"] > li > a')
    state_urls={url['href'].split('/')[-1].split('.')[0]:'https:' + url['href'] for url in urls}
    return state_urls

In [4]:
source = "https://www.city-data.com/articles/"
#tabs_by_category
response = requests.get(source).text
soup = BeautifulSoup(response, 'html.parser')
state_urls=get_stateurl(source,soup)

In [13]:

for state_page_name, state_url in tqdm(state_urls.items()):
    filename = './cache/' + state_page_name + '.json'
    if  not exists(filename):
        with requests.get(state_url, stream=True) as x:
            response=x.text
            soup = BeautifulSoup(response, 'html.parser')
    # read the site pages
            site_urls = soup.select('div[id="listing"] > div[class="col-md-4"] > a')
            site_dict = {}
            for site_url in site_urls:
                key = 'https://www.city-data.com' + site_url['href']
                with requests.get('https://www.city-data.com' + site_url['href'], stream=True) as r:# to avoid error
                    value=r.text
                    site_dict[key] = value
                time.sleep(0.1) # add a gap to prevent being blocked
        filename = './cache/' + state_page_name + '.json'
        with open(filename, 'w+') as json_file:
            json.dump(site_dict, json_file, indent=4)
        time.sleep(300)
   
 
        

 10%|█         | 8/77 [14:06<2:01:41, 105.82s/it]


ChunkedEncodingError: ('Connection broken: IncompleteRead(0 bytes read, 8096 more expected)', IncompleteRead(0 bytes read, 8096 more expected))

In [6]:
def get_cache_site_page(state_page_name, site_url):
    # helper function: get response of a site page, return None if not exist
    filename = './cache/' + state_page_name + '.json'
    try:
        with open(filename) as json_file:
            site_dict = json.load(json_file)
            response = site_dict[site_url]
    except:
        return None
    
    return response

def get_site_instance(state_page_name, site_url):
    # get the site instance for a site page from a state_page_name and site_url
    response = get_cache_site_page(state_page_name, site_url)
    soup = BeautifulSoup(response, 'html.parser')
    
    state = state_page_name.translate(str.maketrans('', '', digits))
    title = soup.select('h1[class="city"] > span')[0].contents[0]
    paragraphs = [p.contents[0] for p in soup.select('div[class="well"] > p') if not p.find_all()]
    description = ' '.join(paragraphs)
    if title.count(' - ') == 2:
        name = title.split(' - ')[0]
    elif title.count(', ') == 2:
        name = title.split(', ')[0]
    else:
        name = title

    touristsite = CityData(name.replace('"', ""),
                              state.replace('"', ""), 
                              title.replace('"', ""), 
                              description.replace('"', ""))
    
    return touristsite
def get_cache_sites_for_state_page(state_page_name):
    # get a list of site urls given the state page
    filename = './cache/' + state_page_name + '.json'
    try:
        with open(filename) as json_file:
            site_dict = json.load(json_file)
    except:
        return None
    
    site_list = list(site_dict.keys())
    return site_list

In [7]:
#write data to csv files:
# write to the csv file
print("--- START WRITING FILES TO DOC DATA ---")
count=0
for state_page_name, state_url in tqdm(state_urls.items()):
    count+=1
    site_list = get_cache_sites_for_state_page(state_page_name)
    print(f"Writing sites from <{state_page_name}> to csv file...")
    filename = './data/' + state_page_name + '_documents.csv'
    if not exists(filename) and count<=8:
        for site_url in tqdm(site_list):
            try:
                site = get_site_instance(state_page_name, site_url)
                site.write_single_csv()
                site.write_whole_csv()
            except:
                pass

--- START WRITING FILES TO DOC DATA ---


  0%|          | 0/77 [00:00<?, ?it/s]

Writing sites from <Alaska> to csv file...


  1%|▏         | 1/77 [01:18<1:38:56, 78.11s/it]

Writing sites from <Alabama> to csv file...


...

Writing sites from <California> to csv file...


  9%|▉         | 7/77 [10:25<2:01:25, 104.08s/it]

Writing sites from <California2> to csv file...


100%|██████████| 77/77 [12:44<00:00,  9.93s/it]  

Writing sites from <California3> to csv file...
Writing sites from <California4> to csv file...
Writing sites from <California5> to csv file...
Writing sites from <California6> to csv file...
Writing sites from <California7> to csv file...
Writing sites from <California8> to csv file...
Writing sites from <California9> to csv file...
Writing sites from <California10> to csv file...
Writing sites from <California11> to csv file...
Writing sites from <Colorado> to csv file...
Writing sites from <Colorado2> to csv file...
Writing sites from <Connecticut> to csv file...
Writing sites from <District-of-Columbia> to csv file...
Writing sites from <Delaware> to csv file...
Writing sites from <Florida> to csv file...
Writing sites from <Florida2> to csv file...
Writing sites from <Florida3> to csv file...
Writing sites from <Florida4> to csv file...
Writing sites from <Florida5> to csv file...
Writing sites from <Florida6> to csv file...
Writing sites from <Georgia> to csv file...
Writing site

In [ ]:
def scrape(source_url,soup):
    


In [90]:
state_urls

{'Alaska': 'https://www.city-data.com/articles/Alaska.html',
 'Alabama': 'https://www.city-data.com/articles/Alabama.html',
 'Arkansas': 'https://www.city-data.com/articles/Arkansas.html',
 'Arizona': 'https://www.city-data.com/articles/Arizona.html',
 'Arizona2': 'https://www.city-data.com/articles/Arizona2.html',
 'Arizona3': 'https://www.city-data.com/articles/Arizona3.html',
 'California': 'https://www.city-data.com/articles/California.html',
 'California2': 'https://www.city-data.com/articles/California2.html',
 'California3': 'https://www.city-data.com/articles/California3.html',
 'California4': 'https://www.city-data.com/articles/California4.html',
 'California5': 'https://www.city-data.com/articles/California5.html',
 'California6': 'https://www.city-data.com/articles/California6.html',
 'California7': 'https://www.city-data.com/articles/California7.html',
 'California8': 'https://www.city-data.com/articles/California8.html',
 'California9': 'https://www.city-data.com/articles/